In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 27727276
paper_name = 'payen_dunham_2016' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [9]:
original_data = pd.read_excel('raw_data/journal.pgen.1006339.s008.xlsx', sheet_name='Sheet1', skiprows=8)

In [10]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5632 x 14


In [11]:
original_data.head()

,Name,MM1N-Phosphate,MM2N-Phosphate,CEN-Phosphate,2micron-Phosphate,MM1N-Sulfate,MM2N-Sulfate,CEN-Sulfate,2micron-Sulfate,MM1N-Glucose,MM2N-Glucose,CEN-Glucose,2micron-Glucose,# replicates
0,YAL001C,NaN,0.007918,-0.08260,-0.005923,NaN,-0.064626,0.016187,0.042277,NaN,-0.075990,NaN,0.035646,8
1,YAL002W,NaN,-0.016999,-0.01310,NaN,NaN,-0.005073,0.051772,NaN,NaN,0.008071,0.014010,NaN,6
2,YAL003W,-0.084682,-0.007208,-0.01205,-0.089589,-0.081194,-0.012135,-0.020084,-0.100065,-0.039218,-0.009811,-0.018207,-0.046407,12
3,YAL004W,0.012654,0.006390,NaN,NaN,-0.017815,-0.013990,NaN,NaN,-0.137926,0.011175,NaN,NaN,6
4,YAL005C,-0.002357,-0.042974,-0.00235,NaN,0.002517,NaN,-0.177540,NaN,-0.105089,NaN,0.121381,NaN,7


In [12]:
original_data['orf'] = original_data['Name'].astype(str)

In [13]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [14]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [15]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Name, MM1N-Phosphate, MM2N-Phosphate, CEN-Phosphate, 2micron-Phosphate, MM1N-Sulfate, MM2N-Sulfate, CEN-Sulfate, 2micron-Sulfate, MM1N-Glucose, MM2N-Glucose, CEN-Glucose, 2micron-Glucose, # replicates, orf]
Index: []


In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['MM1N-Phosphate','MM2N-Phosphate','MM1N-Sulfate','MM2N-Sulfate','MM1N-Glucose','MM2N-Glucose']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(5632, 6)

# Prepare the final dataset

In [20]:
data = original_data.copy()

In [21]:
dataset_ids = [16195, 16199, 16210, 16197, 16196, 16198]
datasets = datasets.reindex(index=dataset_ids)

In [22]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [23]:
data.head()

dataset_id,16195,16199,16210,16197,16196,16198
data_type,value,value,value,value,value,value
orf,,,,,,
YAL001C,NaN,0.007918,NaN,-0.064626,NaN,-0.075990
YAL002W,NaN,-0.016999,NaN,-0.005073,NaN,0.008071
YAL003W,-0.084682,-0.007208,-0.081194,-0.012135,-0.039218,-0.009811
YAL004W,0.012654,0.006390,-0.017815,-0.013990,-0.137926,0.011175
YAL005C,-0.002357,-0.042974,0.002517,NaN,-0.105089,NaN


## Subset to the genes currently in SGD

In [26]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 16


In [27]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [28]:
data.head()

,dataset_id,16195,16199,16210,16197,16196,16198
,data_type,value,value,value,value,value,value
gene_id,orf,,,,,,
1,YAL001C,NaN,0.007918,NaN,-0.064626,NaN,-0.075990
2,YAL002W,NaN,-0.016999,NaN,-0.005073,NaN,0.008071
3,YAL003W,-0.084682,-0.007208,-0.081194,-0.012135,-0.039218,-0.009811
1863,YAL004W,0.012654,0.006390,-0.017815,-0.013990,-0.137926,0.011175
4,YAL005C,-0.002357,-0.042974,0.002517,NaN,-0.105089,NaN


# Normalize

In [29]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [30]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [31]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [32]:
data_all.head()


,dataset_id,16195,16199,16210,16197,16196,16198,16195,16199,16210,16197,16196,16198
,data_type,value,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,
1,YAL001C,NaN,0.007918,NaN,-0.064626,NaN,-0.075990,NaN,0.292940,NaN,-1.862701,NaN,-1.817783
2,YAL002W,NaN,-0.016999,NaN,-0.005073,NaN,0.008071,NaN,-0.695176,NaN,-0.052073,NaN,0.255222
3,YAL003W,-0.084682,-0.007208,-0.081194,-0.012135,-0.039218,-0.009811,-1.452869,-0.306915,-1.470815,-0.266787,-0.620117,-0.185742
1863,YAL004W,0.012654,0.006390,-0.017815,-0.013990,-0.137926,0.011175,0.145105,0.232363,-0.388552,-0.323172,-2.210413,0.331787
4,YAL005C,-0.002357,-0.042974,0.002517,NaN,-0.105089,NaN,-0.101336,-1.725291,-0.041348,NaN,-1.681383,NaN


# Print out

In [33]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [34]:
from IO.save_data_to_db3 import *

In [35]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/6 [00:00<?, ?it/s]

Deleting all datasets for PMID 27727276...
Inserting the new data...


100%|██████████| 6/6 [00:51<00:00,  8.56s/it]

Updating the data_modified_on field...
